In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

In [2]:
#load the csv file stored in the folder
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lon,Max Tem,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Albany,42.6001,-73.9662,74.26,66,2,4.21,US,1659532511
1,1,Puyo,-1.4667,-77.9833,67.24,94,75,0.00,EC,1659532605
2,2,Ruzomberok,49.0748,19.3075,81.00,29,6,3.76,SK,1659532494
3,3,Vuktyl,63.8567,57.3094,79.23,34,6,9.31,RU,1659532606
4,4,Busselton,-33.6500,115.3333,54.32,73,100,17.74,AU,1659532606


# Humidity Heat Map

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# create heat map
locations = city_data_df[["Lat", "Lon"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()

#add the layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

In [6]:
#drop data where weather is not ideal
idealWeatherDF = city_data_df.loc[
    (city_data_df["Max Tem"]>70) & (city_data_df["Max Tem"]<80) & \
    (city_data_df["Wind Speed"]<10) & \
    (city_data_df["Cloudiness"] ==0)
].dropna()

#display
idealWeatherDF.head()

,City_ID,City,Lat,Lon,Max Tem,Humidity,Cloudiness,Wind Speed,Country,Date
77,77,Bethel,41.3712,-73.4140,78.49,77,0,0.00,US,1659532461
88,88,Poum,-20.2333,164.0167,74.68,86,0,9.01,NC,1659532640
97,97,Makakilo City,21.3469,-158.0858,73.72,78,0,6.91,US,1659532646
130,130,Novoagansk,61.9449,76.6625,70.03,47,0,9.82,RU,1659532658
141,141,Hilo,19.7297,-155.0900,70.25,94,0,5.75,US,1659532489


#  Hotel Map

In [9]:
hotel_df= idealWeatherDF[["City", "Country", "Lat", "Lon"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Lat,Lon,Hotel Name
77,Bethel,US,41.3712,-73.4140,
88,Poum,NC,-20.2333,164.0167,
97,Makakilo City,US,21.3469,-158.0858,
130,Novoagansk,RU,61.9449,76.6625,
141,Hilo,US,19.7297,-155.0900,
157,East London,ZA,-33.0153,27.9116,
180,Batesville,US,35.7698,-91.6410,
197,Ouro Fino,BR,-22.2831,-46.3692,
204,Vila Franca Do Campo,PT,37.7167,-25.4333,
215,Waynesboro,US,38.0685,-78.8895,


In [12]:
#perform hotel search
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}
#loop through the df using iterrows
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    long = row["Lon"]
    
    #use the lat and long to se the locations value in the parameters
    params["location"] = f"{lat}, {long}"
    
    #use the google apis places base url to search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    #make request and print the url
    name_address = requests.get(base_url, params = params).json()
    
    #see what the api is going to return
    try:
        #populate the hotel name
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print(f"Error finding lodging in {row['City']}. Skipping...")
        pass
    
hotel_df

Error finding lodging in Poum. Skipping...
Error finding lodging in Novoagansk. Skipping...


,City,Country,Lat,Lon,Hotel Name
77,Bethel,US,41.3712,-73.4140,Courtyard by Marriott Danbury
88,Poum,NC,-20.2333,164.0167,
97,Makakilo City,US,21.3469,-158.0858,Marriott's Ko Olina Beach Club
130,Novoagansk,RU,61.9449,76.6625,
141,Hilo,US,19.7297,-155.0900,Hilo Hawaiian Hotel
157,East London,ZA,-33.0153,27.9116,Tu Casa
180,Batesville,US,35.7698,-91.6410,"Holiday Inn Express & Suites Batesville, an IH..."
197,Ouro Fino,BR,-22.2831,-46.3692,Hotel Fazenda Ouro Park
204,Vila Franca Do Campo,PT,37.7167,-25.4333,Pestana Bahia Praia
215,Waynesboro,US,38.0685,-78.8895,Holiday Inn Express & Suites Waynesboro-Route ...


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [14]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))